<a href="https://colab.research.google.com/github/jgarnicaa/GenAI_Poetry/blob/main/gpt2_finetuning_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning gpt-2 to poem generation

In this notebook we're going to fine tune gpt-2 small with our custom poetry dataset (english and spanish)

Why use this architecture?
- Have multilanguage support
- Small LLM model to deploy
- Easy to train

We're going to use MLFlow to compare different models

Model taken from: https://huggingface.co/openai-community/gpt2

To use MLFlow:

> mlflow ui --backend-store-uri file:///tmp/mlflow --default-artifact-root s3://genaipoetry-bucket/mlflow/artifacts



> mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root s3://genaipoetry-bucket/mlflow/artifacts --host 0.0.0.0 --port 5000







In [1]:
!pip install mlflow boto3 transformers datasets dvc dvc[s3]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
#Libraries

import mlflow
import mlflow.pytorch
import os
import boto3
import shutil
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, concatenate_datasets
from google.colab import drive

In [2]:
## AWS Enviroment config to save MLFlow Experiments in bucket s3
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://s3.amazonaws.com"

In [3]:
# Config S3
os.environ["MLFLOW_ARTIFACT_URI"] = "s3://genaipoetry-bucket/mlflow-artifacts/"
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlflow.db"
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [4]:
drive.mount('/content/drive')
os.makedirs("results", exist_ok=True)

Mounted at /content/drive


In [5]:
!git clone https://github.com/jgarnicaa/GenAI_Poetry.git ## Clone to recover dataset from DVC

Cloning into 'GenAI_Poetry'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 109 (delta 30), reused 87 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 239.37 KiB | 4.60 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [6]:
!cd GenAI_Poetry/ && dvc pull ##Recover dataset

Fetching
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
Querying remote cache:   0% 0/1 [00:00<?, ?files/s]
Querying remote cache:   0% 0/1 [00:00<?, ?files/s{'info': ''}]
Querying remote cache: 100% 1/1 [00:00<00:00,  9.62files/s{'info': ''}]
                                                                       
Fetching from s3:   0% 0/5 [00:00<?, ?file/s]
Fetching from s3:   0% 0/5 [00:00<?, ?file/s{'info': ''}]
Fetching from s3:   0% 0/4 [00:00<?, ?file/s{'info': ''}]

  0% 0.00/10.9M [00:00<?, ?B/s]

  0% 0.00/10.9M [00:00<?, ?B/s{'info': ''}]

  1% 101k/10.9M [00:00<00:20, 567kB/s{'info': ''}]

  3% 352k/10.9M [00:00<00:10, 1.05MB/s{'info': ''}]

 11% 1.21M/10.9M [00:00<00:03, 2.82MB/s{'info': ''}]

 28% 3.06M/10.9M [00:00<00:01, 7.11MB/s{'info': ''}]

 57% 6.23M/10.9M [00:00<00:00, 13.7MB/s{'info': ''}]

 89% 9.73M/10.9M [00:00<00:00, 17.5MB/s{'info': ''}]

                                                    
Fetching from s3:  25% 1/4

In [7]:
#Load dataset

df_es=load_dataset("csv", data_files={"train": "/content/GenAI_Poetry/data/ES_corpus/ES_poetry_cleaned.csv"} )
df_en=load_dataset("csv", data_files={"train": "/content/GenAI_Poetry/data/EN_corpus/EN_poetry_cleaned.csv"} )

dataset=DatasetDict({"train": concatenate_datasets([df_es["train"], df_en["train"]])}) ## Merge EN and ES dataset

dataset=dataset.remove_columns(["Unnamed: 0", "Title", "Author", "Unnamed: 0.1", "Poet", "Tags"]) ## Use only Poem corpus, drop rest of info


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") ## DEfine tokenizer and model from gpt2
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['Poem'], truncation=True, padding='max_length')

    # Las etiquetas son iguales a los input_ids (desplazados para la predicción del siguiente token)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

In [10]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [11]:

tokenized_dataset= dataset.map(tokenize_function, batched=True) ## Tokenize dataset

print(tokenized_dataset["train"][0]) ## Print first example

Map:   0%|          | 0/25563 [00:00<?, ? examples/s]

{'Poem': '\n\n\r\nEntre nosotros crece la ropa en las mañanas\n\r\nse atraviesan mil veces los oficios\n\r\nnos mueven los deberes\n\r\nel futuro\n\r\nlas cosas.\n\n\n\r\nPor si no fuera mucho alguien propone la medida\n\r\npara que no te vayas\n\r\n-dicen-\n\r\nes necesario el regateo.\n\r\nPero tus manos son mi tiempo\n\r\ny no quiero jugar a detener la boca y los abrazos.\n\r\nTe irás más tarde\n\r\n-dicen-\n\r\nsi encuentro la mesura\n\r\npero deseo tu cuerpo y este día\n\r\neste preciso cielo\n\r\nla película de hoy\n\r\nla cama próxima\n\r\ntu sudor y tu piel ahora en la tarde.\n\n\n\r\nNo voy a retener mis frases ni mi aliento\n\r\nno me quiero tragar ni un poco de silencio\n\r\nni uno solo de los consentimientos.\n\n\n\r\n¿Por qué la luz a medias?\n\r\n¿Para que no te vayas cuando te irás?\n\r\nNunca se mete el sol antes de tiempo\n\r\ny se pone lo mismo en días nublados.\n\r\nYo quiero tu cobija hasta que quieras\n\r\nte doy mientras\n\r\nmis ansias, mis costumbres,\n\r\nmis r

In [12]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/GenAIPoetry/checkpoints',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=6,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    warmup_steps=500,
    weight_decay=0.01,
    report_to="mlflow"
)

In [13]:
mlflow.set_experiment("FineTuning-GPT2-Poetry")

2025/02/12 09:46:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/02/12 09:46:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/content/mlruns/1', creation_time=1739353581282, experiment_id='1', last_update_time=1739353581282, lifecycle_stage='active', name='FineTuning-GPT2-Poetry', tags={}>

In [14]:
## Training with MLFLow logging
with mlflow.start_run():
    trainer=Trainer(model=model,
                    args=training_args,
                    train_dataset=tokenized_dataset["train"])

    trainer.train(resume_from_checkpoint=True)

    mlflow.log_param("epochs", training_args.num_train_epochs)
    mlflow.log_param("batch size", training_args.per_device_train_batch_size)

    model.save_pretrained('./fine-tuned-gpt2-poetry')

    mlflow.log_artifacts('./fine-tuned-gpt2-poetry', artifact_path="models") #save S3

    mlflow.pytorch.log_model(model, artifact_path="models", registered_model_name="GPT2-Poetry")


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Step,Training Loss


2025/02/12 09:47:18 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/02/12 09:47:44 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-01-14; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'google-spark-connect'}
2025/02/12 09:47:44 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local ver

In [15]:
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(experiment_ids=["1"])
for run in runs:
    print(f"Run ID: {run.info.run_id}")

Run ID: 544a11a246a244bb8dd6d0839b874d17


In [16]:
logged_model = "runs:/544a11a246a244bb8dd6d0839b874d17/models"
# Cargar el modelo
model = mlflow.pytorch.load_model(logged_model)


In [41]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model.to(device)

def generate_poem(prompt, max_length=100, temperature=1.2, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=temperature, #1.2 #0.7
        top_p=top_p, #0.95 #0.9
        do_sample=True
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

print("English poem: ",generate_poem("Under the moonlight,"))

print("Spanish poem: ",generate_poem("Bajo el sol"))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Device: cpu


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English poem:  Under the moonlight, I walk through the field, my arms and legs trembling, like little children at the start of a song.






As for us, each was created to sing, who have the right to be free of  any pain? The song of song and of love is my song,  that I sing them in silence so as to not be understood. But they know me. I know  they love
Spanish poem:  Bajo el sol que amarte mi oye

y entra mía, y vienes amantes;

y la tarde que desvanecen.

De un poco aún, que cual me entretienes,

y de mi amor no me deseo alguna vez.

¿Entonces, Dios,

en este sol desde sus man


In [45]:
from google.colab import files
files.download("/content/mlflow.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
!zip -r /content/drive/MyDrive/model.zip /content/fine-tuned-gpt2-poetry

  adding: content/fine-tuned-gpt2-poetry/ (stored 0%)
  adding: content/fine-tuned-gpt2-poetry/config.json (deflated 52%)
  adding: content/fine-tuned-gpt2-poetry/generation_config.json (deflated 24%)
  adding: content/fine-tuned-gpt2-poetry/model.safetensors (deflated 7%)


In [46]:
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>